# Attention Vizualizer

Howdy! Regardless of what shows up below, click Kernel --> Restart and Run All to get started.

Mouse over each Glimpse to view the post-windowing input to the network. It might be kind of slow, since it's a network round trip, so be patient. Enjoy!

In [ ]:
from viz import *

Setting everything up!


Loading BokehJS ...

/home/sychen23/.local/lib/python3.5/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
The bokeh.charts API has moved to a separate 'bkcharts' package.

This compatibility shim will remain until Bokeh 1.0 is released.
After that, if you want to use this API you will have to install
the bkcharts package explicitly.

  warn(message)


['/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py', 'true', 'false', 'true', 'false', 'true', 'true', 'model_runs/test_rect_list/classify_log.csv', 'model_runs/test_rect_list/classifymodel_0.ckpt', 'model_runs/test_rect_list/classifymodel_', 'model_runs/test_rect_list/zzzdraw_data_5000.npy', 'false', 'true', 'false', 'false', 'true']
classification.shape, the batch_size, z_size:  (1, 9)
classification.shape, the batch_size, z_size:  (1, 9)
classification.shape, the batch_size, z_size:  (1, 9)
classification.shape, the batch_size, z_size:  (1, 9)
classification.shape, the batch_size, z_size:  (1, 9)
classification.shape, the batch_size, z_size:  (1, 9)
classification.shape, the batch_size, z_size:  (1, 9)
classification.shape, the batch_size, z_size:  (1, 9)
classification.shape, the batch_size, z_size:  (1, 9)
classification.shape, the batch_size, z_size:  (1, 9)
classification.shape, the batch_size, z_size:  (1, 9)
classification.shape, the batch_size, z_size:  (1, 9)
cla